Finetuning and evaluation of the finetuned model

In [1]:
pip install fsspec==2024.10.0

In [2]:
!pip cache purge

Files removed: 0


In [1]:
!pip install -q -U datasets
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes
! pip install -q -U rouge_score
! pip install -q -U langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 33.8 MB/s eta 0:00:00


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00


In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.2 MB/s eta 0:00:00


In [4]:
from evaluate import load

### Imports

In [5]:
import pandas as pd
import json
import torch
import os
import time

from datasets import load_dataset, Dataset, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print

# LangChain Imports
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate  #, LLMChain
from langchain.chains import RetrievalQA
import textwrap

### Global Constants

In [6]:
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os

MODEL_PATH = "/content/drive/MyDrive/Text2SQL/FTClean/"
MODEL_DIR = "sql_gptq_training_30/"
output_dir = os.path.join(MODEL_PATH, MODEL_DIR)
os.makedirs(output_dir, exist_ok=True)

print(f"Directory created or already exists: {output_dir}")

Directory created or already exists: /content/drive/MyDrive/Text2SQL/FTClean/sql_gptq_training_30/

In [8]:
model_id = "TheBloke/Llama-2-7B-GPTQ"
MODEL_PATH = "/content/drive/MyDrive/Text2SQL/FTClean/"
MODEL_DIR = "sql_gptq_training"+"_30/"
checkpoint_name ="SQL_llama2_gptq_7b_peftv1_20241213_054152"

In [9]:
print(checkpoint_name)

SQL_llama2_gptq_7b_peftv1_20241213_054152

In [10]:
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"
LLMRES_PKL_FILE="sql_test_llmres.pkl"

In [13]:
import os

DATA_PATH = "/content/drive/MyDrive/Text2SQL/Data/"
PKL_DIR = "test/"
LLMRES_PKL_FILE = "sql_test_llmres.pkl"

target_dir = os.path.join(DATA_PATH, PKL_DIR)
os.makedirs(target_dir, exist_ok=True)
file_path = os.path.join(target_dir, LLMRES_PKL_FILE)
with open(file_path, "wb") as f:
    pass
print(f"Empty file {LLMRES_PKL_FILE} successfully created at {file_path}")

Empty file sql_test_llmres.pkl successfully created at 
/content/drive/MyDrive/Text2SQL/Data/test/sql_test_llmres.pkl

### Common Functions

In [11]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [12]:
rouge = load("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

In [14]:
def gen_eval_res(test_df):
  eval_res = []
  time_res = []
  row_no = 1
  for row in test_df.itertuples(index=False):
    start_time = perf_counter()
    text = row.text
    llm_res = pipe(text)[0]['generated_text']
    parse_res = parse(llm_res)
    end_time = perf_counter()
    eval_res.append(parse_res)
    time_res.append(end_time-start_time)
    print(f"*** Row {row_no} Done ***")
    row_no += 1

  test_df['eval_res'] = eval_res
  test_df['eval_time'] = time_res

### Load and Check Data

In [15]:
dataset = load_from_disk(DATA_PATH + DS_DIR)

In [16]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [17]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [18]:
display(test_df.head(2))
display(test_df.shape)

,response,question,context,__index_level_0__,text
0,"SELECT T2.name, COUNT(*) FROM race AS T1 JOIN ...",Show the name of track and the number of races...,"CREATE TABLE track (name VARCHAR, track_id VAR...",429,### Instruction:\n You are a powerful text-...
1,"SELECT T3.Shop_Name, T2.Carrier FROM stock AS ...",Show names of shops and the carriers of device...,"CREATE TABLE shop (Shop_Name VARCHAR, Shop_ID ...",2907,### Instruction:\n You are a powerful text-...


(454, 5)

In [19]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=quantization_config_loading,
                                             device_map="auto")

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Llama-2-7B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bia

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Finetuning

In [20]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["k_proj","o_proj","q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model.config.use_cache = False

trainable params: 8,388,608 || all params: 270,798,848 || trainable%: 3.0977


In [21]:
print(model.get_memory_footprint()/1e9)

3.92742528

In [22]:
OUT_DIR = "sql_gptq_training"

In [23]:
args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir=OUT_DIR,
        overwrite_output_dir=True,
        optim="adamw_hf",
        save_strategy="epoch",
        report_to="none")

In [24]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    warmup_steps=2,
    max_steps=5,
    optim="adamw_hf",
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    peft_config=config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [26]:
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.180700
2,2.013000
3,1.913800
4,1.992100
5,1.717700
6,1.571800
7,1.377200
8,1.186300
9,1.062800
10,0.906600


In [27]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Fine-tuning completed. Checkpoint saved at: {output_dir}")

Fine-tuning completed. Checkpoint saved at: /content/drive/MyDrive/Text2SQL/FTClean/sql_gptq_training_30/

### Load Model and Setup Trainer

In [32]:
output_dir = MODEL_PATH + MODEL_DIR
print(output_dir)

/content/drive/MyDrive/Text2SQL/FTClean/sql_gptq_training_30/

In [33]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoPeftModelForCausalLM.from_pretrained(
          output_dir,
          low_cpu_mem_usage=True,
          return_dict=True,
          torch_dtype=torch.float16,
          device_map="cuda",
      )

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Llama-2-7B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj

In [34]:
print(model.get_memory_footprint()/1e9)

3.92742528

In [35]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=180,
    do_sample = True,
    temperature=0.1,
    top_k=4,
    repetition_penalty=1.2
)

llm = HuggingFacePipeline(pipeline=pipe)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

#### Check Model on Sample Test Question

In [36]:
ID = 10

In [37]:
test_df['text'][ID]

'### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    Show ids for all students who have advisor 1121.\n    ### Context:\n    CREATE TABLE Student (StuID VARCHAR, Advisor VARCHAR)\n    ### Response:\n    '

In [38]:
display(test_df['response'][ID])

'SELECT StuID FROM Student WHERE Advisor = 1121'

In [39]:
text = test_df['text'][ID]
llm_res = pipe(text)[0]['generated_text']
parse_res = parse(llm_res)
print(parse_res)

SELECT StuID FROM Student WHERE Advisor = "1121"

In [40]:
eval_start_time = perf_counter()
gen_eval_res(test_df)
eval_end_time = perf_counter()
print('Evaluation Took: ', round(eval_end_time-eval_start_time, 2))

*** Row 1 Done ***

*** Row 2 Done ***

*** Row 3 Done ***

*** Row 4 Done ***

*** Row 5 Done ***

...

*** Row 451 Done ***

*** Row 452 Done ***

*** Row 453 Done ***

*** Row 454 Done ***

Evaluation Took:  10090.59

In [61]:
test_df.to_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)
test_df2=pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)



In [58]:
!pip install fuzzywuzzy


In [60]:
from fuzzywuzzy import fuzz
import pandas as pd
def calculate_similarity(row):
    return fuzz.ratio(row['response'], row['eval_res'])
test_df2['similarity_score'] = test_df2.apply(calculate_similarity, axis=1)
sorted_df = test_df2.sort_values(by='similarity_score', ascending=False)
sorted_df = sorted_df.reset_index(drop=True)
display(sorted_df)



,response,question,context,__index_level_0__,text,eval_res,eval_time,similarity_score
0,SELECT T1.company_name FROM Third_Party_Compan...,Which company started the earliest the mainten...,CREATE TABLE Third_Party_Companies (company_na...,1879,### Instruction:\n You are a powerful text-...,SELECT T1.company_name FROM Third_Party_Compan...,12.362753,100
1,"SELECT Investor, COUNT(*) FROM entrepreneur GR...",What are the investors of entrepreneurs and th...,CREATE TABLE entrepreneur (Investor VARCHAR),1372,### Instruction:\n You are a powerful text-...,"SELECT Investor, COUNT(*) FROM entrepreneur GR...",4.718956,100
2,"SELECT AVG(age), gender FROM Person GROUP BY g...",What is the average age for each gender?,"CREATE TABLE Person (gender VARCHAR, age INTEGER)",2543,### Instruction:\n You are a powerful text-...,"SELECT AVG(age), gender FROM Person GROUP BY g...",29.240591,100
3,SELECT COUNT(*) FROM Faculty,How many faculty do we have?,CREATE TABLE Faculty (Id VARCHAR),3809,### Instruction:\n You are a powerful text-...,SELECT COUNT(*) FROM Faculty,29.258721,100
4,SELECT DISTINCT LOCATION FROM track,What are distinct locations where tracks are l...,CREATE TABLE track (LOCATION VARCHAR),419,### Instruction:\n You are a powerful text-...,SELECT DISTINCT LOCATION FROM track,3.581223,100
...,...,...,...,...,...,...,...,...
449,SELECT T3.area_code FROM contestants AS T1 JOI...,List the area codes in which voters voted both...,"CREATE TABLE votes (contestant_number VARCHAR,...",4344,### Instruction:\n You are a powerful text-...,SELECT T1.area_code FROM area_code_state AS T2...,29.277313,15
450,"SELECT T2.allergytype, COUNT(*) FROM Has_aller...",Show all allergy type with number of students ...,CREATE TABLE Has_allergy (allergy VARCHAR); CR...,290,### Instruction:\n You are a powerful text-...,SELECT T1.allergypeople FROM Allergy_type AS T...,29.270885,13
451,"SELECT T3.Pilot_name, T2.Model FROM pilot_reco...",Show the names of pilots and models of aircraf...,CREATE TABLE pilot_record (Aircraft_ID VARCHAR...,1244,### Instruction:\n You are a powerful text-...,"SELECT T1.Pilot_Name, T2.Model FROM pilot AS T...",29.280978,12
452,SELECT t3.title FROM authors AS t1 JOIN author...,Which paper is published in an institution in ...,"CREATE TABLE authorship (authid VARCHAR, paper...",1717,### Instruction:\n You are a powerful text-...,SELECT T1.paperid FROM papers AS T2 JOIN autho...,29.274655,11
